# Gathering Data

The data on the website are not in one place, but spread out through several pages on the 
Southeast regional Climate Center website. Thus, my first step was not within python but gathering
the data and placing it into a manageable format.

North Carolina is separated into 3 regions Morehead City, Raleigh/Durham, and Wilmington. However, all of the weather stations that the SRCC uses in North Carolina are not all contained in these 3 regions. Several other regions that are placed under other states have regions that overlap into North Carolina, thus one must click on each of the regions within NC and all of the neighboring states regions in order to gather the data for all of the stations that are located in North Carolina. Then I had to go through and click on each location within the region, select the appropriate monthly rain data and copy and paste this data into an excel file as an individual sheet.

In addition, my excel data file contains surrounding locations from these neighboring states that are close to North Carolina. We will use these points as exogenous datapoints to see any relationship between these locations and the locations within NC 

# Data Wrangling

#### Step 1

I imported the necessary modules and then load the excel spreadsheet that I used to collect all of the data from the website.

In [2]:
import os
import getpass
import pandas as pd
import math
import numpy as np
from math import radians, sin, cos, asin, sqrt
import csv

curr_dir = os.path.abspath('')
app_root = curr_dir if os.path.basename(curr_dir) != "src" else os.path.dirname(curr_dir)

file = os.path.join(app_root,'data','NC Monthly Precipitation Data.xlsx')
NCdata = pd.ExcelFile(file)

if getpass.getuser() == "rainfalld":  # docker daemon
    home = os.path.expanduser("~")
    destdir = home                    # /var/cache/rainfall-predictor
else:
    destdir = os.path.join(app_root,'data')      # non-docker stay in repository


#### Step 2
I needed to bring all of the separate sheets in excel together into a single dataframe. Thus, this function uses a for loop to parse out each sheet from the excel file. Once I had the sheet, all of the months were in separate columns. The function takes all of the months and places them next to the year then places the rainfall amounts for each month in the next column. The function merges the resulting dataframe into the blank dataframe on the year and the month columns with an outer join in order to catch all the data from both the original and merging dataframe. 

In [19]:
blank = pd.DataFrame()
def datachunks(s, e, df): #s and e stand for the beginning and end of the chunk you want
    for i in range(s,e):
        to_merge = NCdata.parse(i, skiprows=[0,1], usecols=[0,1,2,3,4,5,6,7,8,9,10,11,12]) #first two rows in the data were titles
        to_merge = to_merge.dropna() #removes two rows from the data that were labeled as NaN and not needed
        to_merge = to_merge.set_index('Year') #set the index to year to remove the following 3 rows
        to_merge = to_merge.drop(['Mean','Max', 'Min'])
        to_merge = to_merge.reset_index() #resets the index so that the dataframe can be melted on Year
        to_merge1 = pd.melt(to_merge, id_vars=['Year'], value_vars=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug',
        'Sep','Oct','Nov','Dec'], var_name='Month') 
        to_merge1.iloc[:,2] = pd.to_numeric(to_merge1.iloc[:,2], errors = 'coerce')
        if i == 0:
            df = to_merge1
        else:
            df = pd.merge(df, to_merge1, on = ['Year','Month'], how = 'outer')
    return df

ncdata = datachunks(0,234, blank)
ncdata.tail()

,Year,Month,value_x,value_y,value_x,value_y,value_x,value_y,value_x,value_y,...,value_x,value_y,value_x,value_y,value_x,value_y,value_x,value_y,value_x,value_y
1951,1864,Dec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1952,1865,Dec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1953,1866,Dec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1954,1867,Dec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1955,1868,Dec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Step 3
Column names to be placed on top of the dataframe

In [20]:
colnames = ['Year', 'Month']
names = NCdata.sheet_names
ncdata.columns = colnames + names
ncdata.head()

,Year,Month,"Raleigh, NC","Greensboro, NC","Fayetteville, NC","Albemarle, NC","Arcola, NC","Asheboro, NC","Burlington, NC","Carthage, NC",...,"ROGERSVILLE 1 NE, TN","SODDY DAISY-MOWBRAY MTN, TN","SPRING CITY, TN","TAZEWELL, TN","TOWNSEND 5S, TN","KING, NC","ABINGDON 3S, VA","WISE 1SE, VA","John Kerr Dam, VA","Emporia, VA"
0,1887,Jan,3.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1888,Jan,3.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1889,Jan,6.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1890,Jan,0.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1891,Jan,3.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Step 4
In order to sort the data based on the year and month I needed to first convert the columns to datetime

In [21]:
ncdata["Month"] = pd.to_datetime(ncdata.Month, format='%b', errors='coerce').dt.month
ncdata["Year"] = pd.to_datetime(ncdata.Year, format='%Y', errors='coerce').dt.year
print(ncdata[['Year', 'Month']].head())


   Year  Month
0  1887      1
1  1888      1
2  1889      1
3  1890      1
4  1891      1


In [22]:
#this sorts the data based on year then month
ncdata_sorted = ncdata.sort_values(['Year','Month'])
ncdata_sorted.tail(12)

,Year,Month,"Raleigh, NC","Greensboro, NC","Fayetteville, NC","Albemarle, NC","Arcola, NC","Asheboro, NC","Burlington, NC","Carthage, NC",...,"ROGERSVILLE 1 NE, TN","SODDY DAISY-MOWBRAY MTN, TN","SPRING CITY, TN","TAZEWELL, TN","TOWNSEND 5S, TN","KING, NC","ABINGDON 3S, VA","WISE 1SE, VA","John Kerr Dam, VA","Emporia, VA"
132,2019,1,3.43,4.01,2.77,4.60,NaN,4.43,NaN,NaN,...,4.62,7.10,8.27,5.54,5.32,4.65,5.10,4.17,3.38,NaN
265,2019,2,4.26,5.89,2.05,2.97,NaN,5.86,NaN,4.67,...,12.22,10.00,14.42,14.67,9.64,6.00,9.22,10.51,5.70,5.61
398,2019,3,3.17,3.19,2.66,2.62,NaN,3.86,NaN,NaN,...,4.74,4.77,6.98,4.94,5.51,2.35,3.13,3.78,3.63,3.90
531,2019,4,6.36,4.16,4.89,6.60,NaN,5.42,NaN,NaN,...,5.55,7.75,6.29,3.58,7.97,4.39,4.52,5.28,3.23,3.98
664,2019,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.82,NaN,4.34,5.10,5.47,2.22,2.82,4.39,NaN,1.23
797,2019,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
930,2019,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1063,2019,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1196,2019,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1329,2019,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Step 5
Instead of separate columns for year and month the following code creates a single Date column and sets it as the index. The index is a string because datetime does not allow for dates without a day; however, having a day listed in the datetime would not be reasonable in this dataset because these are monthly totals of rainfall not occurring on a single day. 

In [23]:
ncdata_sorted['Year'] = ncdata_sorted.Year.apply(str)
ncdata_sorted['Month'] = ncdata_sorted.Month.apply(str)
ncdata_sorted['Date'] = ncdata_sorted['Month'] + '-' + ncdata_sorted['Year']
ncdata1 = ncdata_sorted.set_index('Date')
ncdata1 = ncdata1.drop(['Year', 'Month'], axis=1)
ncdata1.head()

,"Raleigh, NC","Greensboro, NC","Fayetteville, NC","Albemarle, NC","Arcola, NC","Asheboro, NC","Burlington, NC","Carthage, NC","Chapel Hill, NC","Clayton, NC",...,"ROGERSVILLE 1 NE, TN","SODDY DAISY-MOWBRAY MTN, TN","SPRING CITY, TN","TAZEWELL, TN","TOWNSEND 5S, TN","KING, NC","ABINGDON 3S, VA","WISE 1SE, VA","John Kerr Dam, VA","Emporia, VA"
Date,,,,,,,,,,,,,,,,,,,,,
1-1857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2-1857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3-1857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4-1857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5-1857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Step 6 - removing impossible data
I gathered this data in May 2019; thus, it was impossible to have any totals from months that had not happened yet; therefore, I removed them

In [24]:
ncdata1 = ncdata1.drop(['5-2019', '6-2019', '7-2019', '8-2019','9-2019','10-2019','11-2019','12-2019'], axis=0)
ncdata1.tail(12)

,"Raleigh, NC","Greensboro, NC","Fayetteville, NC","Albemarle, NC","Arcola, NC","Asheboro, NC","Burlington, NC","Carthage, NC","Chapel Hill, NC","Clayton, NC",...,"ROGERSVILLE 1 NE, TN","SODDY DAISY-MOWBRAY MTN, TN","SPRING CITY, TN","TAZEWELL, TN","TOWNSEND 5S, TN","KING, NC","ABINGDON 3S, VA","WISE 1SE, VA","John Kerr Dam, VA","Emporia, VA"
Date,,,,,,,,,,,,,,,,,,,,,
5-2018,5.12,3.77,2.68,3.37,NaN,7.56,NaN,2.98,NaN,4.05,...,5.23,7.40,3.55,5.94,4.27,9.85,4.10,5.39,8.03,3.55
6-2018,1.98,2.53,4.31,3.11,NaN,1.79,NaN,NaN,NaN,3.55,...,5.26,7.35,9.26,7.22,5.27,2.74,3.39,6.51,5.82,5.03
7-2018,4.92,6.04,6.36,5.83,NaN,2.69,NaN,NaN,NaN,7.25,...,2.13,3.35,7.39,3.10,4.19,2.29,3.37,5.08,5.63,5.23
8-2018,6.62,6.97,5.75,5.37,NaN,9.26,NaN,9.43,NaN,7.36,...,4.85,6.15,7.39,5.51,6.98,6.72,9.23,3.46,4.64,6.39
9-2018,7.97,9.17,15.54,12.83,NaN,14.11,NaN,NaN,NaN,10.65,...,7.18,11.25,11.61,4.61,7.61,7.00,7.03,6.70,4.70,4.98
10-2018,4.58,7.22,2.22,6.87,NaN,5.24,NaN,NaN,NaN,3.93,...,4.05,2.80,2.89,2.63,4.18,7.40,3.58,3.30,5.05,4.49
11-2018,7.10,6.46,5.31,8.22,NaN,9.18,NaN,NaN,NaN,6.29,...,5.88,8.10,7.62,5.17,NaN,5.83,5.25,5.21,6.61,6.29
12-2018,6.23,6.99,6.00,8.43,NaN,6.41,NaN,NaN,NaN,9.24,...,6.72,7.11,9.49,7.04,8.75,6.54,5.12,6.53,4.85,4.44
1-2019,3.43,4.01,2.77,4.60,NaN,4.43,NaN,NaN,NaN,4.74,...,4.62,7.10,8.27,5.54,5.32,4.65,5.10,4.17,3.38,NaN


#### Step 7
There was a lot of missing data from several locations. Due to this I created the following for loop in order to see which rows (corresponding to a single month) had at least 70% of data. Since the function len() counts missing data while the method .count() does not, I used these two functions to figure out the percentage that each row has and made it a column in the dataframe called 'percent_number'

I found that from January 1956-present all had data from at least 70% of the locations. 

In [25]:
lop = []
for i in ncdata1.index:
    l = len(ncdata1.loc[i])
    c = ncdata1.loc[i].count()
    percent = (c/l)*100
    if i == 0:
        lop = [percent]
    else:
        lop = lop + [percent]
ncdata1['percent_number'] = lop
ncdata1.percent_number[ncdata1.percent_number >= 80].head(20)

Date
8-1980     80.341880
10-1980    80.341880
11-1980    80.341880
1-1981     80.769231
2-1981     80.341880
3-1981     81.623932
4-1981     80.769231
5-1981     80.341880
6-1981     80.341880
7-1981     80.341880
9-1981     81.196581
10-1981    80.341880
4-1982     81.196581
7-1982     81.196581
9-1982     80.769231
10-1982    80.769231
11-1982    81.196581
2-1983     80.769231
3-1983     80.341880
4-1983     80.341880
Name: percent_number, dtype: float64

#### Step 8
Since I had the Date column as my index and I didn't want to remove it, I created a new index row called row_number. I used the row number to figure out which row 1-1956 was located at in order to create the dataframe that includes only data from 1-1956-present

In [26]:
nl = [i for i in range(1948)]
ncdata1['row_number'] = nl 
ncdata1.row_number.loc['1-1980'] # provides the row which Jan 1956 is located

1476

In [27]:
ncdata_80 = ncdata1[ncdata1.row_number >= 1476]
ncdata_80.head()

,"Raleigh, NC","Greensboro, NC","Fayetteville, NC","Albemarle, NC","Arcola, NC","Asheboro, NC","Burlington, NC","Carthage, NC","Chapel Hill, NC","Clayton, NC",...,"SPRING CITY, TN","TAZEWELL, TN","TOWNSEND 5S, TN","KING, NC","ABINGDON 3S, VA","WISE 1SE, VA","John Kerr Dam, VA","Emporia, VA",percent_number,row_number
Date,,,,,,,,,,,,,,,,,,,,,
1-1980,4.39,4.00,3.77,4.87,4.01,4.65,4.43,4.95,4.08,3.57,...,NaN,6.06,NaN,NaN,4.22,NaN,4.76,5.34,75.213675,1476
2-1980,1.91,1.77,1.77,0.84,1.29,1.55,1.82,1.21,2.05,2.03,...,NaN,2.43,NaN,NaN,1.48,1.15,1.45,1.27,77.777778,1477
3-1980,5.87,5.04,5.93,11.39,5.59,6.33,5.69,NaN,6.28,5.19,...,NaN,7.51,NaN,NaN,5.84,4.32,4.02,3.24,74.786325,1478
4-1980,1.97,3.24,3.61,3.49,1.94,1.86,2.79,NaN,2.32,2.94,...,NaN,3.37,NaN,NaN,3.51,4.03,3.24,2.73,79.487179,1479
5-1980,2.33,3.23,1.77,5.14,2.24,4.36,3.86,NaN,4.50,2.30,...,NaN,3.79,NaN,NaN,4.08,2.88,4.01,4.66,78.205128,1480


In [28]:
#this drops the percent number column that is no longer needed.
ncdata_80 = ncdata_80.drop(['percent_number'],axis=1)

#### Step 9
The dataset also has some missing data from when the location began gathering data to the present. For example, even though Raleigh, NC has been gathering data from 1-1956 until the present there was one month in October of 2000 where the monthly total was not recorded. Thus the following function fills in missing data that are contained within the locations. This function finds the months with missing data and fills them in by averaging the rainfall totals from the previous year, previous month and next month. If the any of these points are not available either the function uses the data from two years prior, two months prior, or two months after. If just one of these is not available it ignores the NaN and averages the other two, otherwise it keeps the datapoint as NaN. Thus, this function does not get rid of all missing values, but fills in the missing values as long as there is adequate data to do so.

In [29]:
def missingfill(df, column):
    missing = df.index[df[column].isnull()]
    if len(missing) > 0:
        for n in missing:
            moth = df.loc[n].row_number # finds the row index for the missing data point
            if ((moth >= 1488) & (moth <= 1945)): #must be a year after 1-1956 (rownumber=1188) otherwise it is impossible to have the previous year's data to gather from
                ly = moth - 12 #the previous year's row number
                lyrd = df[[column]][df['row_number'] == ly] # the previous year's rainfall amount as a dataframe
                lyrd1 = lyrd[column][0] #separates the value of the previous year's dataframe to just the rainfall amount
                lm = moth - 1 # the next 6 lines perform the same as the previous 3 except for previous month and following month
                lmrd = df[[column]][df['row_number'] == lm]
                lmrd1 = lmrd[column][0]
                nm = moth + 1
                nmrd = df[[column]][df['row_number'] == nm]
                nmrd1 = nmrd[column][0]
                if ((math.isnan(lyrd1)) & (moth >= 1500)): # if the previous year was not available, go back 2 years
                    twy = moth - 24
                    twyrd = df[[column]][df['row_number'] == twy]
                    lyrd1 = twyrd[column][0]
                if (math.isnan(lmrd1)): #if the previous month was not available, go back 2 months
                    lm = moth - 2
                    lmrd = df[[column]][df['row_number'] == lm]
                    lmrd1 = lmrd[column][0]
                if (math.isnan(nmrd1)): #if the next month was not available, go forward 2 months
                    nm = moth + 2
                    nmrd = df[[column]][df['row_number'] == nm]
                    nmrd1 = nmrd[column][0]
                newpoint = np.nanmean([lyrd1,lmrd1,nmrd1]) #finds the average of the 3 values 
                df.loc[n,column] = newpoint #places the value into the missing data slot
    return(df)


#### Step 10 
performs the function defined in the previous cell and applies the function to every column. The runtime warning just means that np.nanmean has only nan values and thus is returning a nan value again, which is fine. 

In [30]:
for i in ncdata_80.columns:
    ncdata_80 = missingfill(ncdata_80, i)
ncdata_80.info()
    


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: Mean of empty slice


<class 'pandas.core.frame.DataFrame'>
Index: 472 entries, 1-1980 to 4-2019
Columns: 235 entries, Raleigh, NC to row_number
dtypes: float64(234), int64(1)
memory usage: 890.2+ KB


#### Step 11

Drop the row_number column since it is no longer needed, and make a csv file from the dataframe. 

In [31]:
# drops the row_number column since it is no longer needed. 
ncdata_80 = ncdata_80.drop(['row_number'],axis=1)
ncdatarein = ncdata_80.reset_index()
# #converts Date to datetime object
ncdatarein['Date'] = pd.to_datetime(ncdatarein['Date'])
alldatadf = ncdatarein.set_index('Date')
alldatadf.head()

,"Raleigh, NC","Greensboro, NC","Fayetteville, NC","Albemarle, NC","Arcola, NC","Asheboro, NC","Burlington, NC","Carthage, NC","Chapel Hill, NC","Clayton, NC",...,"ROGERSVILLE 1 NE, TN","SODDY DAISY-MOWBRAY MTN, TN","SPRING CITY, TN","TAZEWELL, TN","TOWNSEND 5S, TN","KING, NC","ABINGDON 3S, VA","WISE 1SE, VA","John Kerr Dam, VA","Emporia, VA"
Date,,,,,,,,,,,,,,,,,,,,,
1980-01-01,4.39,4.00,3.77,4.87,4.01,4.65,4.43,4.95,4.08,3.57,...,5.15,NaN,NaN,6.06,NaN,NaN,4.22,NaN,4.76,5.34
1980-02-01,1.91,1.77,1.77,0.84,1.29,1.55,1.82,1.21,2.05,2.03,...,1.28,NaN,NaN,2.43,NaN,NaN,1.48,1.15,1.45,1.27
1980-03-01,5.87,5.04,5.93,11.39,5.59,6.33,5.69,NaN,6.28,5.19,...,6.50,NaN,NaN,7.51,NaN,NaN,5.84,4.32,4.02,3.24
1980-04-01,1.97,3.24,3.61,3.49,1.94,1.86,2.79,NaN,2.32,2.94,...,3.54,NaN,NaN,3.37,NaN,NaN,3.51,4.03,3.24,2.73
1980-05-01,2.33,3.23,1.77,5.14,2.24,4.36,3.86,NaN,4.50,2.30,...,2.80,NaN,NaN,3.79,NaN,NaN,4.08,2.88,4.01,4.66


In [32]:
alldatadf = alldatadf.drop(['Raleigh AP, NC', 'Greensboro, NC', ' WILMINGTON 7 N, NC','LUMBERTON, NC','MYRTLE BEACH, SC','CHARLOTTE DOUGLAS AIRPORT, NC','GRNVL SPART INTL AP, SC','PICKENS, SC',' MT. MITCHELL, NC',' Caesars Head Area, SC'], axis=1)

#### Step 12

Providing distance data from all locations to all other locations using LAT/LONG coordinates. First, the functionto calculate distance between two points on the globe

In [33]:
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])    
    dlon = lon2 - lon1
    dlat = lat2 - lat1    
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    return 2 * 6371 * asin(sqrt(a))

In [35]:
latlong = pd.read_csv(os.path.join(app_root,'data','latlong.csv'))
latlongsplit = latlong.iloc[0].apply(str.split, sep=',')


In [36]:
latlongdf = pd.DataFrame(latlongsplit)
latlongdf = latlongdf.drop(['Unnamed: 0','Raleigh AP, NC', 'Greensboro, NC', ' WILMINGTON 7 N, NC','LUMBERTON, NC','MYRTLE BEACH, SC','CHARLOTTE DOUGLAS AIRPORT, NC','GRNVL SPART INTL AP, SC','PICKENS, SC',' MT. MITCHELL, NC',' Caesars Head Area, SC'])

In [37]:
def distance_loc(df):
    didf = pd.DataFrame(columns=df.index)
    row = {}
    for index in df.index:
        lat1, long1 = float(df.loc[index][0][0]), float(df.loc[index][0][1])
        for i in df.index:
            lat2, long2 = float(df.loc[i][0][0]), float(df.loc[i][0][1])
            dist = haversine(long1, lat1, long2, lat2)
            row[i] = dist
        didf = didf.append(row, ignore_index=True)
    return(didf)
distdf = distance_loc(latlongdf)
distdf.head()
    

,"Raleigh, NC","Fayetteville, NC","Albemarle, NC","Arcola, NC","Asheboro, NC","Burlington, NC","Carthage, NC","Chapel Hill, NC","Clayton, NC","Dunn, NC",...,"ROGERSVILLE 1 NE, TN","SODDY DAISY-MOWBRAY MTN, TN","SPRING CITY, TN","TAZEWELL, TN","TOWNSEND 5S, TN","KING, NC","ABINGDON 3S, VA","WISE 1SE, VA","John Kerr Dam, VA","Emporia, VA"
0,0.000000,94.585842,140.085045,85.792266,94.339063,62.855960,82.357628,24.091159,39.774281,66.211053,...,383.408144,584.886253,547.807265,433.620424,448.437640,147.774483,299.806111,360.119566,91.189561,143.134766
1,94.585842,0.000000,125.911792,161.191610,113.510236,128.420869,60.206872,98.844170,78.111573,38.492773,...,403.550011,577.136588,547.797864,452.278187,448.515057,192.981486,335.857671,397.298667,181.613934,219.228482
2,140.085045,125.911792,0.000000,225.368465,52.177467,106.776872,71.125584,120.265039,161.137251,144.442984,...,278.775025,455.080224,423.324418,326.968370,323.712079,103.481463,219.111882,279.347812,219.856405,280.862479
3,85.792266,161.191610,225.368465,0.000000,177.274348,132.741643,166.828150,105.465420,83.102366,123.265782,...,450.769592,662.304609,622.463285,500.748900,524.100499,213.328129,360.149586,417.417784,44.762293,58.859511
4,94.339063,113.510236,52.177467,177.274348,0.000000,54.666260,54.116409,72.025241,122.813858,117.851109,...,297.732791,491.341157,455.630417,347.532403,355.960562,80.435479,223.597227,285.125827,168.452756,231.039743


In [38]:
distdf2 = distdf
distdf2.index = distdf.columns

In [39]:
distdf2.head(10)

,"Raleigh, NC","Fayetteville, NC","Albemarle, NC","Arcola, NC","Asheboro, NC","Burlington, NC","Carthage, NC","Chapel Hill, NC","Clayton, NC","Dunn, NC",...,"ROGERSVILLE 1 NE, TN","SODDY DAISY-MOWBRAY MTN, TN","SPRING CITY, TN","TAZEWELL, TN","TOWNSEND 5S, TN","KING, NC","ABINGDON 3S, VA","WISE 1SE, VA","John Kerr Dam, VA","Emporia, VA"
"Raleigh, NC",0.000000,94.585842,140.085045,85.792266,94.339063,62.855960,82.357628,24.091159,39.774281,66.211053,...,383.408144,584.886253,547.807265,433.620424,448.437640,147.774483,299.806111,360.119566,91.189561,143.134766
"Fayetteville, NC",94.585842,0.000000,125.911792,161.191610,113.510236,128.420869,60.206872,98.844170,78.111573,38.492773,...,403.550011,577.136588,547.797864,452.278187,448.515057,192.981486,335.857671,397.298667,181.613934,219.228482
"Albemarle, NC",140.085045,125.911792,0.000000,225.368465,52.177467,106.776872,71.125584,120.265039,161.137251,144.442984,...,278.775025,455.080224,423.324418,326.968370,323.712079,103.481463,219.111882,279.347812,219.856405,280.862479
"Arcola, NC",85.792266,161.191610,225.368465,0.000000,177.274348,132.741643,166.828150,105.465420,83.102366,123.265782,...,450.769592,662.304609,622.463285,500.748900,524.100499,213.328129,360.149586,417.417784,44.762293,58.859511
"Asheboro, NC",94.339063,113.510236,52.177467,177.274348,0.000000,54.666260,54.116409,72.025241,122.813858,117.851109,...,297.732791,491.341157,455.630417,347.532403,355.960562,80.435479,223.597227,285.125827,168.452756,231.039743
"Burlington, NC",62.855960,128.420869,106.776872,132.741643,54.666260,0.000000,83.377798,39.995737,101.371222,115.271612,...,322.225060,529.605431,490.345266,372.443346,391.643722,85.216934,237.084444,297.280521,116.639772,182.053038
"Carthage, NC",82.357628,60.206872,71.125584,166.828150,54.116409,83.377798,0.000000,70.943340,93.301656,73.330652,...,344.784924,526.198519,494.190253,393.910565,394.496684,134.457038,275.653084,337.101921,172.121503,225.043388
"Chapel Hill, NC",24.091159,98.844170,120.265039,105.465420,72.025241,39.995737,70.943340,0.000000,61.400111,78.474640,...,359.339721,561.379484,523.961690,409.548974,424.663786,124.146240,276.213878,336.723095,102.410041,160.662985
"Clayton, NC",39.774281,78.111573,161.137251,83.102366,122.813858,101.371222,93.301656,61.400111,0.000000,40.663681,...,418.143017,613.482178,578.418160,468.251091,478.718289,185.109194,337.065316,397.801868,106.644851,141.408233
"Dunn, NC",66.211053,38.492773,144.442984,123.265782,117.851109,115.271612,73.330652,78.474640,40.663681,0.000000,...,415.146618,599.476607,567.434479,464.728045,467.709926,191.496771,340.580955,402.031153,146.624696,180.959040


In [40]:
# args: rdf = rain dataframe
#       ddf = distance df

def missingfillsurrounding(rdf, ddf):
    locwmd = rdf.columns[rdf.isna().any()].tolist()
    for loc in locwmd:
        nbloc = rdf[ddf[[loc]][ddf[loc] <=85].index]
        missing = nbloc.index[nbloc[loc].isnull()]
        if len(missing) >0:
            for m in missing:
                newpt = np.nanmean(nbloc.loc[m])
                rdf.loc[m,loc] = newpt
    return(rdf)
alldatadf_filled = missingfillsurrounding(alldatadf, distdf2)

In [41]:
locations = alldatadf_filled.columns
ncloc = locations[locations.str.endswith('NC')]
valoc = locations[locations.str.endswith('VA')]
scloc = locations[locations.str.endswith('SC')]
galoc = locations[locations.str.endswith('GA')]
tnloc = locations[locations.str.endswith('TN')]
ncdatadf = alldatadf_filled[ncloc]
vadatadf = alldatadf_filled[valoc]
scdatadf = alldatadf_filled[scloc]
gadatadf = alldatadf_filled[galoc]
tndatadf = alldatadf_filled[tnloc]

In [42]:
alldatadf_filled.to_csv(os.path.join(destdir,'rainfalldata.csv'))
distdf2.to_csv(os.path.join(destdir,'distances.csv'))
#ncdatadf.to_csv('ncrainfalldata.csv')
#vadatadf.to_csv('varainfalldata.csv')
#scdatadf.to_csv('scrainfalldata.csv')
#gadatadf.to_csv('garainfalldata.csv')
#tndatadf.to_csv('tnrainfalldata.csv')
#ncdatadf['Raleigh, NC'].to_csv('raleighrain.csv')

In [43]:
# list of target locations = tarloc
# list of exo locations = exoloc
# latitude, longitude df = lldf
def exofind(lldf, tarloc, exoloc):
    tarexoloc = tarloc.append(exoloc)
    tarexoll = lldf.loc[tarexoloc]
    tartoexodist = distance_loc(tarexoll)
    exodistances = tartoexodist[exoloc]
    exodistances.index = tartoexodist.columns
    exodist2 = exodistances.drop(exoloc,axis=0)
    closeexo = exodist2[exodist2 <= 50]
    closeexo1 = closeexo.dropna(how='all')
    closeexo2 = closeexo1.dropna(axis=1,how='all')
    exo = {}
    for i in closeexo2.index:
        ex = closeexo2.loc[i][closeexo2.loc[i].notnull()].index.tolist()
        exo[i]=ex
    return(exo)

In [44]:
distdf2.loc[['CHESNEE 7 WSW, SC', 'CHESTER 1 SE, SC', 'GAFFNEY 6 E, SC', 'LOCKHART, SC'],' MOUNT HOLLY 4 NE, NC'].head()

CHESNEE 7 WSW, SC    78.398450
CHESTER 1 SE, SC     68.131470
GAFFNEY 6 E, SC      62.780155
LOCKHART, SC         69.238312
Name:  MOUNT HOLLY 4 NE, NC, dtype: float64

In [45]:
distdf2.loc[['CHESTERFIELD 3 E, SC',
  ' CHERAW, SC',
  'PAGELAND 9.0 WNW, SC',
  'FORT MILL 4 NW, SC'],'Albemarle, NC'].head()

CHESTERFIELD 3 E, SC    70.019300
 CHERAW, SC             78.619941
PAGELAND 9.0 WNW, SC    67.376230
FORT MILL 4 NW, SC      78.244661
Name: Albemarle, NC, dtype: float64

In [46]:
exoloc = valoc.append(scloc)
exoloc = exoloc.append(galoc)
exoloc = exoloc.append(tnloc)
exogen = exofind(latlongdf,ncloc,exoloc)
exogen

{'Arcola, NC': ['John Kerr Dam, VA'],
 'Henderson 2 NNW, NC': ['John Kerr Dam, VA'],
 'Laurinburg, NC': [' DILLON, SC', ' CHERAW, SC'],
 'Roanoke Rapids, NC': ['Emporia, VA'],
 'Murfreesboro, NC': ['Emporia, VA'],
 'Lumberton Area, NC': [' DILLON, SC'],
 'LONGWOOD, NC': [' LORIS 2 S, SC', 'Myrtle Beach Area, SC'],
 'WHITEVILLE 7 NW, NC': [' LORIS 2 S, SC'],
 'Charlotte Area, NC': ['CATAWBA, SC', 'FORT MILL 4 NW, SC'],
 'Mount Mitchell Area, NC': ['ERWIN 1 W, TN'],
 'ASHEVILLE AIRPORT, NC': ['CAESARS HEAD, SC', 'CLEVELAND 3S, SC'],
 'BANNER ELK, NC': ['ELIZABETHTON, TN',
  'ERWIN 1 W, TN',
  'ROAN MOUNTAIN 3SW, TN'],
 'BEECH MOUNTAIN, NC': ['BRISTOL AP, TN',
  'ELIZABETHTON, TN',
  'ERWIN 1 W, TN',
  'ROAN MOUNTAIN 3SW, TN'],
 'BRYSON CITY 4, NC': ['GATLINBURG 2 SW, TN',
  'MT LECONTE, TN',
  'NEWFOUND GAP, TN',
  ' TOWNSEND 5S, TN'],
 'BREVARD, NC': ['Pickens Area, SC', 'CAESARS HEAD, SC', 'CLEVELAND 3S, SC'],
 'CASAR, NC': ['CHESNEE 7 WSW, SC', 'GAFFNEY 6 E, SC'],
 'COWEETA EXP STATIO

In [47]:
l_val = []
for key,value in exogen.items():
    l_val.append(len(value))
print(np.mean(l_val),max(l_val), len(exogen.keys()), np.sum(l_val))

2.4565217391304346 5 46 113


In [48]:
try:
  %store exogen
except NameError:
  pass

Stored 'exogen' (dict)


In [49]:
from tqdm import tqdm_notebook as tqdm
from itertools import combinations
def exog_combinations_len(exoe):
    lo_dfs = []
    if len(exoe) == 1:
        lo_dfs.append(1)
    if len(exoe) > 1:
        lo_dfs.append(1)
        for ex in exoe:
            lo_dfs.append(1)
        if len(exoe) >2:
            for i in range(2, len(exoe)):
                combolist = list(combinations(exoe,i))
                for c in combolist:
                    lo_dfs.append(1)
    return(lo_dfs)
l_val2 =[]
for key,value in tqdm(exogen.items()):
    lolo = exog_combinations_len(value)
    l_val2.append(len(lolo))
np.sum(l_val2)

318